In [4]:
from datasets import load_dataset

In [5]:
train_data_path = 'data/results/exp6/train/correct_examples.jsonl'
test_data_path = 'data/results/exp6/test/correct_examples.jsonl'

In [6]:
train_dataset = load_dataset(path='json', data_files=train_data_path, split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /Users/ericmelz/.cache/huggingface/datasets/json/default-daefa6a3cdb26f1b/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


In [8]:
test_dataset = load_dataset(path='json', data_files=test_data_path, split='train')

Found cached dataset json (/Users/ericmelz/.cache/huggingface/datasets/json/default-1cb811f9a9d6b30b/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


In [10]:
print(train_dataset[0]['title'])

Given a math problem, generate an answer with a rationale.

Question / answer pairs have the form

Question: ${question}

${answer}
    
Examples:
    
Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?

In May, Natalia sold half as many clips as in April, which is 48/2 = <<48/2=24>>24 clips.
The total number of clips sold in May and April is 24 clips in May + 48 clips in April = <<24+48=72>>72 clips. Answer: \boxed{72}.


Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?

In May Natalia sold half as many clips as she did in April, so in May she sold 48 / 2 = <<48/2=24>>24 clips.
Altogether in April and May Natalia sold 48 + 24 = <<48+24=72>>72 clips. Answer: \boxed{72}.


Question: Natalia sold clips to 48 of her friends in April, and then she sold half as m

## in the train set, how many retrieved questions perfectly match the query question?

In [12]:
import re

In [62]:
def perfect_match_pct():
    """
    Examine each question in the training prompts
    Count the number of examples that perfectly match the target question
    
    Return total_examples, total_matching_examples, total_matching_examples / total_examples
    """
    pattern = r'Question:.*\n'
    total_examples = 0
    total_matching_examples = 0
    
    for i in range(len(train_dataset)):    
        prompt = train_dataset[i]['title']
        questions = re.findall(pattern, prompt)
        if len(questions) != 7:
            print(f'BAD PROMPT ({i}), {len(questions)=}:')
            print(prompt)
            return
        total_examples += 5
        total_matching_examples += len(list(filter(lambda x: questions[-1] == x, questions[1:-1])))

    return total_examples, total_matching_examples, total_matching_examples / total_examples
    

In [64]:
perfect_match_pct()

(22240, 17348, 0.7800359712230216)

## what kind of examples are being retrieved in the test set?  are names or nouns being used over verbs?

In [65]:
print(test_dataset[0]['title'])

Given a math problem, generate an answer with a rationale.

Question / answer pairs have the form

Question: ${question}

${answer}
    
Examples:
    
Question: In a grocery store, Julia bought 2 pieces of Snickers and 3 packs of M&M's. If each piece of Snickers costs $1.5 and a pack of M&M's has the same cost as 2 Snickers, how much is Julia's change if she gave the cashier 2 $10 bills?

Two pieces of snickers amount to $1.5 x 2 = $<<1.5*2=3>>3.
Julia bought 3 packs of M&M's which amount to $3 x 3 = $<<3*3=9>>9.
And the total amount she needs to pay is $3 + $9 = $<<3+9=12>>12.
Julia gave the cashier 2 $10 bills, so her 2 x $10 = $<<2*10=20>>20.
Therefore her change is $20 - $12 = $<<20-12=8>>8. Answer: \boxed{8}.


Question: Frank goes to the store to buy some food. He buys 5 chocolate bars and 2 bags of chips. He hands the cashier $20 and gets $4 back as change. If the chocolate bars each cost $2, how much did each bag of chips cost?

The chocolate bars cost Frank 5 * $2 = $<<5*2=10

### The query question obviously retrieved other questions about food! ^^^